In [1]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict
import glob
import time

import pandas as pd
from pyarrow.lib import ArrowIOError
from tqdm import tqdm
import vaex

print(f"pd.__version__: {pd.__version__}")
print(f"vaex.__file__: {vaex.__file__}")
print(f"vaex.__version__: {vaex.__version__}")

pd.__version__: 0.25.3
vaex.__file__: /home/olga/miniconda3/envs/khtools--encodings-compare-cli--py37/lib/python3.7/site-packages/vaex/__init__.py
vaex.__version__: 1.0.0-beta.6


In [2]:
import sys

In [3]:
sys.path

['/home/olga/code/kh-analysis--olgabot/qfo-kmers/notebooks',
 '/home/olga/miniconda3/envs/khtools--encodings-compare-cli--py37/lib/python37.zip',
 '/home/olga/miniconda3/envs/khtools--encodings-compare-cli--py37/lib/python3.7',
 '/home/olga/miniconda3/envs/khtools--encodings-compare-cli--py37/lib/python3.7/lib-dynload',
 '',
 '/home/olga/miniconda3/envs/khtools--encodings-compare-cli--py37/lib/python3.7/site-packages',
 '/home/olga/miniconda3/envs/khtools--encodings-compare-cli--py37/lib/python3.7/site-packages/IPython/extensions',
 '/home/olga/.ipython']

In [4]:
ll /home/olga/miniconda3/envs/khtools--encodings--compare-cli/lib/python3.8/site-packages/v*

/home/olga/miniconda3/envs/khtools--encodings--compare-cli/lib/python3.8/site-packages/vaex-2.5.0.dist-info:
total 32
-rw-r--r-- 1 olga    4 Jan  7 14:16 INSTALLER
-rw-r--r-- 1 olga 1087 Jan  7 14:16 LICENSE.txt
-rw-r--r-- 1 olga 8331 Jan  7 14:16 METADATA
-rw-r--r-- 1 olga  715 Jan  7 14:16 RECORD
-rw-r--r-- 1 olga    5 Jan  7 14:16 top_level.txt
-rw-r--r-- 1 olga   93 Jan  7 14:16 WHEEL

/home/olga/miniconda3/envs/khtools--encodings--compare-cli/lib/python3.8/site-packages/vaex_arrow:
total 24
-rw-r--r-- 1 olga 4081 Jan  7 14:30 convert.py
-rw-r--r-- 1 olga 2733 Jan  7 14:30 dataset.py
-rw-r--r-- 1 olga 4018 Jan  7 14:30 export.py
-rw-r--r-- 1 olga    0 Jan  7 14:30 __init__.py
-rw-r--r-- 1 olga  679 Jan  7 14:30 opener.py
drwxr-xr-x 2 olga 4096 Jan  7 14:30 __pycache__/
-rw-r--r-- 1 olga   52 Jan  7 14:30 _version.py

/home/olga/miniconda3/envs/khtools--encodings--compare-cli/lib/python3.8/site-packages/vaex_arrow-0.4.2.dist-info:
total 28
-rw-r--r-- 1 olga   58 Jan  7 14:30 entry_p

## Check disk-mounted storage

In [5]:
cd  ~/data_sm/kmer-hashing/quest-for-orthologs/analysis/2019/intermediates

/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/analysis/2019/intermediates


In [6]:
ls -1 | wc -l

1798


# Download OMA GETHOGs benchmark results
This had the most number of ortholog pairs of all the services submitted to the [Quest for Orthologs' benchmarking service - Gene Ontology Conservation test](https://orthology.benchmarkservice.org/cgi-bin/gateway.pl?f=CheckResults&p1=10f28e6c09fd6acc1a90ade6)

In [7]:
# ! wget https://orthology.benchmarkservice.org/raw//GO/GO.83ebff415cfec35b9ae1888e.tsv.gz
# ! zcat GO.83ebff415cfec35b9ae1888e.tsv.gz | head

In [8]:
oma = pd.read_csv('GO.83ebff415cfec35b9ae1888e.tsv.gz', sep='\t', skiprows=2, header=None)
oma.columns = ['protein_id1', 'protein_id2', 'go_similarity']
print(oma.shape)
oma.head()

(927900, 3)


,protein_id1,protein_id2,go_similarity
0,Q5JCX3,P07650,0.373544
1,Q5JCX3,P19971,0.467737
2,Q5JDA9,O67379,1.000000
3,Q5JF32,Q9DEA3,0.460952
4,Q5JF32,P17918,0.575735


### Reverse protein id1 and id2 to get all relations for joining

In [9]:
oma_reversed = oma.rename(columns={'protein_id1': 'protein_id2', 'protein_id2': 'protein_id1'})
print(oma_reversed.shape)
oma_reversed.head()



(927900, 3)


,protein_id2,protein_id1,go_similarity
0,Q5JCX3,P07650,0.373544
1,Q5JCX3,P19971,0.467737
2,Q5JDA9,O67379,1.000000
3,Q5JF32,Q9DEA3,0.460952
4,Q5JF32,P17918,0.575735


In [10]:
oma_concat = pd.concat([oma, oma_reversed], ignore_index=True)
# oma_concat = oma_concat.set_index('protein_id1')
print(oma_concat.shape)
oma_concat.head()

(1855800, 3)


/home/olga/miniconda3/envs/khtools--encodings-compare-cli--py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,go_similarity,protein_id1,protein_id2
0,0.373544,Q5JCX3,P07650
1,0.467737,Q5JCX3,P19971
2,1.000000,Q5JDA9,O67379
3,0.460952,Q5JF32,Q9DEA3
4,0.575735,Q5JF32,P17918


In [11]:
# oma_concat_subset = oma_concat.query('protein_id1 in @df.protein_id1 and '
#                                      'protein_id2 in @df.protein_id2')
# print(oma_concat_subset.shape)
# oma_concat_subset.head()

# Read parquet intermediates from local

In [12]:
scratch_folder = '/home/olga/scratch/kmer-hashing/quest-for-orthologs/analysis/2019/intermediates'

In [13]:
arrow_folder = '/home/olga/scratch/kmer-hashing/quest-for-orthologs/analysis/2019/intermediates-vaex-arrow'
! mkdir $arrow_folder

mkdir: cannot create directory ‘/home/olga/scratch/kmer-hashing/quest-for-orthologs/analysis/2019/intermediates-vaex-arrow’: File exists


In [14]:
cd $scratch_folder

/home/olga/scratch/kmer-hashing/quest-for-orthologs/analysis/2019/intermediates


In [15]:
pwd

'/home/olga/scratch/kmer-hashing/quest-for-orthologs/analysis/2019/intermediates'

In [16]:
ls -1 | wc -l

1798


In [17]:
ls -lha | head

total 346G
drwxr-xr-x 2 olga czb 116K Jan  7 09:09 ./
drwxr-xr-x 4 olga czb 4.0K Jan  7 11:32 ../
-rw-r--r-- 1 olga czb 6.2M Sep  6  2018 GO.83ebff415cfec35b9ae1888e.tsv.gz
-rw-r--r-- 1 olga czb 225M Dec 30 14:54 sp__A5HEI1-2__SCC2-2_ARATH.parquet
-rw-r--r-- 1 olga czb 198M Dec 30 19:38 sp__A5HEI1-3__SCC2-3_ARATH.parquet
-rw-r--r-- 1 olga czb 198M Dec 31 02:49 sp__A5PHT0-2__Y1448-2_ARATH.parquet
-rw-r--r-- 1 olga czb 196M Jan  2 23:30 sp__A5Z2X3__FUTB2_CAEEL.parquet
-rw-r--r-- 1 olga czb 197M Jan  1 00:36 sp__A8MQA3-2__PP330-2_ARATH.parquet
-rw-r--r-- 1 olga czb 197M Dec 30 07:15 sp__F4HYV5__NAC26_ARATH.parquet
ls: write error


## Opening parquet directly with vaex works, but resulting df isn't workable

`df.head()` gives the following traceback:

```pytb
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-29-c42a15b2c7cf> in <module>
----> 1 df.head()

~/miniconda3/envs/khtools--encodings--compare-cli/lib/python3.8/site-packages/vaex/dataframe.py in head(self, n)
   3397     def head(self, n=10):
   3398         """Return a shallow copy a DataFrame with the first n rows."""
-> 3399         return self[:min(n, len(self))]
   3400 
   3401     def tail(self, n=10):

~/miniconda3/envs/khtools--encodings--compare-cli/lib/python3.8/site-packages/vaex/dataframe.py in __getitem__(self, item)
   4552                 assert stop != -1
   4553                 stop = stop+1  # +1 to make it inclusive
-> 4554             df = self.trim()
   4555             df.set_active_range(start, stop)
   4556             return df.trim()

~/miniconda3/envs/khtools--encodings--compare-cli/lib/python3.8/site-packages/vaex/dataframe.py in trim(self, inplace)
   3782         :rtype: DataFrame
   3783         '''
-> 3784         df = self if inplace else self.copy()
   3785         if self._index_start == 0 and self._index_end == self._length_original:
   3786             return df

~/miniconda3/envs/khtools--encodings--compare-cli/lib/python3.8/site-packages/vaex/dataframe.py in copy(self, column_names, virtual)
   4852                     column = self.columns[name]
   4853                     if not isinstance(column, ColumnSparse):
-> 4854                         df.add_column(name, column, dtype=self._dtypes_override.get(name))
   4855                 elif name in self.virtual_columns:
   4856                     if virtual:  # TODO: check if the ast is cached

~/miniconda3/envs/khtools--encodings--compare-cli/lib/python3.8/site-packages/vaex/dataframe.py in add_column(self, name, data, dtype)
   5819         #     self._length_original = len(data)
   5820         #     self._index_end = self._length_unfiltered
-> 5821         super(DataFrameArrays, self).add_column(name, data, dtype=dtype)
   5822         self._length_unfiltered = int(round(self._length_original * self._active_fraction))
   5823         # self.set_active_fraction(self._active_fraction)

~/miniconda3/envs/khtools--encodings--compare-cli/lib/python3.8/site-packages/vaex/dataframe.py in add_column(self, name, f_or_array, dtype)
   2929                     if len(self) == len(ar):
   2930                         raise ValueError("Array is of length %s, while the length of the DataFrame is %s due to the filtering, the (unfiltered) length is %s." % (len(ar), len(self), self.length_unfiltered()))
-> 2931                 raise ValueError("array is of length %s, while the length of the DataFrame is %s" % (len(ar), self.length_original()))
   2932             # assert self.length_unfiltered() == len(data), "columns should be of equal length, length should be %d, while it is %d" % ( self.length_unfiltered(), len(data))
   2933             valid_name = vaex.utils.find_valid_name(name)

ValueError: array is of length 14810232, while the length of the DataFrame is 32994760
```

In [21]:
# %%time
# df = vaex.open('sp__A5HEI1-2__SCC2-2_ARATH.parquet')

In [22]:
# df.head()

In [31]:
# ls -lha | grep MOUSE | head

In [23]:
# vaex.open_many()

In [24]:
%%time


def extract_protein_id(x):
    return x.split('|')[1]


ts = defaultdict(dict)
dfs = []
for parquet in glob.iglob('*.parquet'):
    prefix = parquet.split('.parquet')[0]
    print(prefix)
    try:
        t0 = time.time()
        df = pd.read_parquet(parquet)
        t1 = time.time()
    except ArrowIOError:
        print(f'Could not read: {parquet}')
    ts[prefix]['pandas__read_parquet'] = t1-t0
    print(ts[prefix])
    
#     t2 = time.time()
#     df['protein_id1'] = df.id1.map(extract_protein_id)
#     df['protein_id2'] = df.id2.map(extract_protein_id)
#     t3 = time.time()
#     ts[prefix]['extract_protein_id'] = t3-t2
    
#     # Merge with OMA output
#     t4 = time.time()
#     df_oma = df.merge(oma, how='outer', on=['protein_id1', 'protein_id2'])
#     t5 = time.time()
#     ts[prefix]['pandas__merge'] = t5-t4
# #     print(df_oma.shape)
# #     df_oma.head()

    t6 = time.time()
    df_vx = vaex.from_pandas(df)
    t7 = time.time()
    ts[prefix]['vaex__from_pandas'] = t7 - t6
    print(ts[prefix])
    
    arrow = f'{arrow_folder}/{prefix}.arrow'

    t8 = time.time()
    df_vx.export_arrow(arrow, progress=True)
    t9 = time.time()
    ts[prefix]['vaex__export_arrow'] = t9 - t8
    print(ts[prefix])
#     del df
#     del df_vx
#     dfs.append(df)
    break
    
times = pd.DataFrame(ts)
print(times.shape)
print(f"times.median(): times.median()")
print(f"times.mean(): times.mean()")
print(f"times.std(): times.std()")
# kmer_similarity = pd.concat(dfs)
# print(kmer_similarity.shape)
# kmer_similarity.head()

tr__A0A2K3CNV6__A0A2K3CNV6_CHLRE
{'pandas__read_parquet': 8.569242477416992}
{'pandas__read_parquet': 8.569242477416992, 'vaex__from_pandas': 517.95427775383}
{'pandas__read_parquet': 8.569242477416992, 'vaex__from_pandas': 517.95427775383, 'vaex__export_arrow': 54.04432487487793}
(3, 1)
times.median(): times.median()
times.mean(): times.mean()
times.std(): times.std()
CPU times: user 8min 47s, sys: 57.1 s, total: 9min 45s
Wall time: 9min 40s


In [38]:
df_vx.head()

#,id1,id2,ksize,jaccard,molecule,index
0,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",3,0.05,protein,0
1,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",4,0,protein,1
2,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",5,0,protein,2
3,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",6,0,protein,3
4,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",7,0,protein,4
5,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",8,0,protein,5
6,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",9,0,protein,6
7,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",10,0,protein,7
8,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",11,0,protein,8
9,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",12,0,protein,9


In [39]:
prefix

'tr__A0A2K3CNV6__A0A2K3CNV6_CHLRE'

## Try exporting to different formats

In [42]:
hdf5 = f'/home/olga/scratch/{prefix}.hdf5'
%time df_vx.export_hdf5(hdf5)

In [43]:
! ls -lha $hdf5

-rw-r--r-- 1 olga czb 9.1G Jan  7 14:02 /home/olga/scratch/tr__A0A2K3CNV6__A0A2K3CNV6_CHLRE.hdf5


In [50]:
%time from_hdf5 = vaex.open(hdf5)
from_hdf5.head()

CPU times: user 7.63 ms, sys: 4.07 ms, total: 11.7 ms
Wall time: 10.5 ms


#,id1,id2,ksize,jaccard,molecule,index
0,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",3,0.05,protein,0
1,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",4,0,protein,1
2,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",5,0,protein,2
3,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",6,0,protein,3
4,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",7,0,protein,4
5,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",8,0,protein,5
6,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",9,0,protein,6
7,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",10,0,protein,7
8,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",11,0,protein,8
9,'tr|A0A2K3CNV6|A0A2K3CNV6_CHLRE Uncharacterized ...,"'tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, isof...",12,0,protein,9


In [44]:
arrow = f'/home/olga/scratch/{prefix}.arrow'
%time df_vx.export_arrow(arrow)

! ls -lha $arrow

CPU times: user 4.45 s, sys: 9.96 s, total: 14.4 s
Wall time: 13 s
-rw-r--r-- 1 olga czb 8.8G Jan  7 14:03 /home/olga/scratch/tr__A0A2K3CNV6__A0A2K3CNV6_CHLRE.arrow


'1.0.0-beta.6'

In [51]:
%time from_arrow = vaex.open(arrow)
from_arrow.head()

CPU times: user 98.4 ms, sys: 24.3 ms, total: 123 ms
Wall time: 121 ms


ValueError: array is of length 17602324, while the length of the DataFrame is 32994760

In [36]:
500 * 2000 / 60 / 60

277.77777777777777

277 hours???

In [34]:
ts

defaultdict(dict,
            {'tr__A0A2K3CNV6__A0A2K3CNV6_CHLRE': {'pandas__read_parquet': 8.137545347213745}})

In [28]:
df.shape

(32994760, 5)

In [39]:
len(dfs)

791

In [35]:
# kmer_similarity = pd.concat(dfs)

In [36]:
print(kmer_similarity.shape)
kmer_similarity.head()

(2540596520, 5)


,id1,id2,ksize,jaccard,molecule
0,sp|O94363|RHB1_SCHPO GTP-binding protein rhb1 ...,"tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, iso...",3,0.083333,protein
1,sp|O94363|RHB1_SCHPO GTP-binding protein rhb1 ...,"tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, iso...",4,0.000000,protein
2,sp|O94363|RHB1_SCHPO GTP-binding protein rhb1 ...,"tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, iso...",5,0.000000,protein
3,sp|O94363|RHB1_SCHPO GTP-binding protein rhb1 ...,"tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, iso...",6,0.000000,protein
4,sp|O94363|RHB1_SCHPO GTP-binding protein rhb1 ...,"tr|A0A024R1R8|A0A024R1R8_HUMAN HCG2014768, iso...",7,0.000000,protein


In [ ]:
f'{len(kmer_similarity.index):,}'

# Convert to vaex for speed

In [ ]:
import vaex as vx

In [ ]:
parquet

In [ ]:
vaex.from_arrow_table(parquet)

# Clean data

## Extract protein ids from full name

### Get unique ids as making a map on unique ids is faster

In [ ]:
%%time
%time id1 = kmer_similarity.id1.unique()
%time id2 = kmer_similarity.id2.unique()


KeyboardInterrupt: 

In [ ]:
all_ids = set(id1) + set(id2)
len(all_ids)

In [38]:
%%time


id1_proteins = {x: extract_protein_id(x) for x in id1}
id2_proteins = {x: extract_protein_id(x) for x in id2}

kmer_similarity['protein_id1'] = kmer_similarity.id1.map(extract_protein_id)
kmer_similarity['protein_id2'] = kmer_similarity.id2.map(extract_protein_id)
# kmer_similarity.head()

KeyboardInterrupt: 

## How many of different proteins are there

In [ ]:
kmer_similarity.protein_id1.nunique()

In [ ]:
kmer_similarity.protein_id2.nunique()

## Check where jaccard is greater than 0

In [ ]:
(kmer_similarity.jaccard > 0).sum()

In [ ]:
kmer_similarity.query('jaccard > 0.8')

In [ ]:
(kmer_similarity.jaccard > 0.8).sum()